In [1]:
import numpy as np
import math
import random

In [2]:
# # RANSAM

def Dis_pt2plane(pts, a, b, c, d):
    """
    Compute the distance from points to the plane
    """
    normal = math.sqrt(a*a+b*b+c*c)
    if normal == 0:
        normal = 1
    
    v = np.array([a,b,c])
    dis = abs(np.dot(pts,v.T)+d)/normal
    return dis

def get_Plane(sampts):
    """
    Compute the equation of the plane
    """
    p1 = sampts[0]
    p2 = sampts[1]
    p3 = sampts[2]
    
    a = ( (p2[1]-p1[1])*(p3[2]-p1[2])-(p2[2]-p1[2])*(p3[1]-p1[1]) )
    b = ( (p2[2]-p1[2])*(p3[0]-p1[0])-(p2[0]-p1[0])*(p3[2]-p1[2]) )
    c = ( (p2[0]-p1[0])*(p3[1]-p1[1])-(p2[1]-p1[1])*(p3[0]-p1[0]) )
    d = ( 0-(a*p1[0]+b*p1[1]+c*p1[2]) )
    
    return a,b,c,d

# def Random3points(points3D, ConfidenceIndex):
#     """
#     Random choose 3 Confidence points
#     """
#     sample_number = 3
#     sample_point_index = random.sample(range(ConfidenceIndex.shape[0]), sample_number)
#     sample_points = np.zeros((sample_number,3))
#     for i in range(sample_number):
#         Confidence_point_index = sample_point_index[i]
#         index = ConfidenceIndex[Confidence_point_index]
#         y = index // points3D.shape[1]
#         x = index % points3D.shape[1]
#         sample_points[i] = points3D[y][x]
#     return sample_points

def Random3points(points3D):
    sample_number = 30
    sample_point_index = random.sample(range(points3D.shape[0]*points3D.shape[1]), sample_number)
    sample_points = np.zeros((3,3))
    num = 0
    for i in range(sample_number):
        index = sample_point_index[i]
        y = index // points3D.shape[1]
        x = index % points3D.shape[1]
        
        point = points3D[y][x]
        if(point[0] != 0 or point[1] != 0 or point[2] != 0):
            sample_points[num] = points3D[y][x]
            num = num + 1
        
        if(num == 3):
            break
    return sample_points

def get_inliner_num(points3D,a,b,c,d,inliner_threshold,Confidence_img):
    """
    Compute the liner points which distance to plane < threshold
    Also get distance from points to the plane (new Depth Image which re-project depth pixels in surface plane)
    """
    inliner_num = 0
    
    dist = Dis_pt2plane(points3D,a,b,c,d)
    inliner_mask = dist < inliner_threshold
    inliner_mask = np.logical_and(inliner_mask, Confidence_img)
    inliner_num = np.sum(inliner_mask)
    return inliner_num, inliner_mask, dist

def RANSAM(points3D, Confidence_img, ransac_iteration = 1000, inliner_threshold = 0.01, last_inliner_num = 30000, Isbreak = True):
    best_inlinernum = -1
    best_inlinernum = 0
    best_plane = np.zeros((1,4))
    best_depthImage = np.zeros((points3D.shape[0],points3D.shape[1]))
    best_plane_mask = np.zeros((points3D.shape[0],points3D.shape[1]))
    best_sampts = np.zeros((3,3))
    
#     print(points3D.shape,points3D[80:90,110])
    for i in range(ransac_iteration):
        sampts = Random3points(points3D)
        a,b,c,d = get_Plane(sampts)
        
        inliner_num, inliner_mask, depthImage = get_inliner_num(points3D,a,b,c,d,inliner_threshold,Confidence_img)
        if(inliner_num > best_inlinernum):
            best_inlinernum = inliner_num
            best_plane = np.array([a,b,c,d])
            best_plane_mask = inliner_mask
            best_depthImage = depthImage
            best_sampts = sampts
          
        if (abs(inliner_num - last_inliner_num) < 1000):
            best_inlinernum = inliner_num
            best_plane = np.array([a,b,c,d])
            best_plane_mask = inliner_mask
            best_depthImage = depthImage
            best_sampts = sampts
            break
        
    #print("Inliner Number\n", best_inlinernum)
    #print("Inliner plane\n", best_plane)
    return best_plane, best_depthImage, best_plane_mask, best_sampts, best_inlinernum

In [3]:
!jupyter nbconvert --to script _RANSAC.ipynb

[NbConvertApp] Converting notebook _RANSAC.ipynb to script
[NbConvertApp] Writing 3890 bytes to _RANSAC.py
